# Kernel Time-varying Graphical Lasso with Real Data

In [65]:
import numpy as np
import pandas as pd
from regainpr.covariance import kernel_latent_time_graph_lasso_
from regainpr.bayesian import wishart_process_; reload(wishart_process_);

In [23]:
# load data
df = pd.read_csv("../regainpr/data/metereological.csv").replace("M", np.NaN) # .fillna(0)

In [24]:
df["temp"] = (df["MAX AT C"].values.astype(float) + df["MIN AT C"].values.astype(float)) / 2.

In [43]:
df['year_month'] = df.DATE.str.split("/").apply(lambda x: "/".join(x[:-1]))

In [61]:
df_temp = df[df.temp.notna()]

In [62]:
idx_ok = [k for k, v in df_temp.groupby("year_month").groups.iteritems() if len(v) > 2]

In [64]:
df_filt = df_temp[df_temp.year_month.isin(idx_ok)]

In [66]:
wp = wishart_process_.WishartProcess(n_iter=500)

In [70]:
X = df_filt[["temp", "SNOW cm", "RAIN cm"]].fillna(0)
y = df_filt.year_month

In [71]:
wp.fit(X, y)

ValueError: axes don't match array